In [196]:
import os
import pandas as pd
from difflib import SequenceMatcher
from pathlib import Path


In [197]:
# to look up similarity scores.
def style_validation_table(row, comparison_df, columns_to_compare):
    styles = [''] * len(row)
    # The 'Sr. No' column is at index 0, so we start from index 1 for predicted columns
    # and map them back to their original column names for comparison_df lookup.
    for i, col_name_display in enumerate(row.index):
        if col_name_display == 'Sr. No':
            continue # Skip styling for Sr. No

        # In this version, col_name_display is already the original column name
        original_col_name = col_name_display

        if original_col_name in columns_to_compare:
            # Get the row index from the display_df (which is the same as actual_df/llm_df index)
            row_idx = row.name # pandas Series has a .name attribute for its index

            # Look up the similarity score from the comparison_df
            # Ensure row_idx is valid for comparison_df before accessing
            if original_col_name in comparison_df.columns and row_idx in comparison_df.index:
                similarity_value = comparison_df.loc[row_idx, original_col_name]

                if pd.notna(similarity_value):
                    if similarity_value == 100:
                        styles[i] = 'color: green;'
                    elif similarity_value < 100: # Any mismatch
                        styles[i] = 'color: red;'
    return styles

In [198]:
base_dir = os.getcwd()  # goes up from app/
pred_file_path = os.path.join(base_dir, '..','combined_output.xlsx')
pred = pd.read_excel(pred_file_path, header=1)

In [199]:
pred.columns

Index(['Remarks/Key notes', 'Billing Date ', 'Month', 'From', 'To',
       'No of Days', 'Day \nkWh', 'Night\nkWh', 'On-Peak', 'Mid-Peak',
       'Off-Peak', 'Super-Off-Peak', 'Only kWh', 'Total kWh', ' kWh per day',
       'kWh %', 'DUOS Capacity Charge', 'Excess Capacity Charge',
       'NC/Max Demand', 'On-Peak\nDemand', 'Mid-Peak\nDemand',
       'Max TOU Demand', 'VAT\n$', 'Franchise tax\n$', 'Sales tax\n%',
       'Franchise tax\n%', 'City of Cullman Tax \n$', 'Alabama State Taxes\n$',
       'Alabama State Taxes\n$.1', 'City of Cullman Tax \n$.1', 'Total tax\n%',
       'Generation/Retail\n $ amount', 'Total\n $ amount\n(With VAT)',
       'Total $ amount\n(Without VAT)', 'Distribution\n $ amount',
       'Total $ amount', 'Generation/Retail rate\n$/kWh',
       'Blended rate\n$/kWh\n(With VAT)', 'Blended rate\n$/kWh\n(Without VAT)',
       'Blended rate\n$/kWh'],
      dtype='object')

In [200]:
acutal_file_path = os.path.join(base_dir, '..','combined_validation_data.xlsx')
actual = pd.read_excel(acutal_file_path, header=1)


In [201]:
(actual['kWh %'] * 100).round(0)

0      3.0
1      3.0
2      4.0
3      3.0
4      2.0
5      3.0
6     11.0
7     13.0
8     13.0
9     14.0
10    15.0
11    16.0
Name: kWh %, dtype: float64

In [207]:

# --------------------------
def novolex_validate_invoices():

    # pred = pd.read_excel('../filled_invoice_novolex.xlsx', header=1)
    # Construct an absolute path from the script
    # base_dir = os.path.dirname(os.path.dirname(__file__))  # goes up from app/
    pred_file_path = os.path.join(base_dir,'..','combined_output.xlsx')
    pred = pd.read_excel(pred_file_path, header=1)

    # base_dir = os.path.dirname(os.path.dirname(__file__))  # goes up from app/
    actual_file_path = os.path.join(base_dir, '..','combined_validation_data.xlsx')
    actual = pd.read_excel(actual_file_path, header=1)
    # actual = pd.read_excel('../novolex_validation_data.xlsx', header=1)

    pred['From'] = pd.to_datetime(pred['From']).dt.strftime('%m/%d/%y')
    pred['To'] = pd.to_datetime(pred['To']).dt.strftime('%m/%d/%y')

    

    # actual['Month'] = actual['Month'].dt.strftime('%b-%y')

    # actual['From'] = pd.to_datetime(actual['From'], format='%d/%m/%Y')
    # actual['From'] = actual['From'].dt.strftime('%d/%m/%y')

    # actual['To'] = pd.to_datetime(actual['To'], format='%d/%m/%Y')
    # actual['To'] = actual['To'].dt.strftime('%d/%m/%y')

    pred['Blended rate\n$/kWh\n(With VAT)'] = pred['Blended rate\n$/kWh\n(With VAT)'].round(2)
    # pred['Total kWh'] = pred['Total kWh'].round(2)
    actual[' kWh per day'] = actual[' kWh per day'].round(2)
    

    
    # pred['Total $ amount\n(Without VAT)'] = pred['Total $ amount\n(Without VAT)'].round(2)

    actual['Blended rate\n$/kWh\n(With VAT)'] = actual['Blended rate\n$/kWh\n(With VAT)'].round(2)
    actual['Blended rate\n$/kWh\n(Without VAT)'] = actual['Blended rate\n$/kWh\n(Without VAT)'].round(2)
    actual['kWh %'] = pd.to_numeric(actual['kWh %'], errors='coerce')
    actual['kWh %'] = (actual['kWh %'] * 100).round(0).astype(int)
    actual['Blended rate\n$/kWh'] = actual['Blended rate\n$/kWh'].round(3)
    

    pred.columns = pred.columns.str.strip()
    actual.columns = actual.columns.str.strip()
    actual.rename(columns={' kWh per day': 'kWh per day'}, inplace=True)
   
    
    # Step 3: Sort the DataFrame by 'Billing Date'
    pred_sorted = pred.sort_values(by='Billing Date')
    actual_sorted = actual.sort_values(by='Billing Date')


    # Optional: Reset index
    pred_sorted = pred_sorted.reset_index(drop=True)
    actual_sorted = actual_sorted.reset_index(drop=True)

    print('Acutl >>>>>>>>>>>>>>>>>', actual['Alabama State Taxes\n$.1'].tolist() )
    print('Pred  >>>>>>>>>>>>>>>>>', pred['Alabama State Taxes\n$.1'].tolist() )

   
    
  
     # Define columns to be compared
    columns_to_compare = ['Billing Date', 'Month', 'From', 'To',
       'No of Days', 'Day \nkWh', 'Only kWh','Night\nkWh', 'Total kWh', 'kWh per day', 'kWh %',
       'DUOS Capacity Charge', 'Excess Capacity Charge', 'VAT\n$',
       'City of Cullman Tax \n$', 'Alabama State Taxes\n$',
       'Alabama State Taxes\n$.1', 'City of Cullman Tax \n$.1',
       'Generation/Retail\n $ amount', 'Total\n $ amount\n(With VAT)',
       'Total $ amount\n(Without VAT)', 'Total $ amount',
       'Blended rate\n$/kWh\n(With VAT)',
       'Blended rate\n$/kWh\n(Without VAT)',
       'Blended rate\n$/kWh']
    
    columns_to_compare = [col for col in columns_to_compare if col in pred_sorted.columns and col in actual_sorted.columns]

    # print(result)
    match_percentage = {}

    # print(result)
    for col in columns_to_compare:
        pred_col = pred_sorted[col]
        actual_col = actual_sorted[col]

        # Compare elements, treating NaNs in same position as a match
        match = (pred_col == actual_col) | (pd.isna(pred_col) & pd.isna(actual_col))

        # print(f"Column: {col}", match)

        # Calculate match percentage
        match_percentage[col] = match.sum() / len(pred_col) * 100
        # print(f"Match Percentage for {col}: {match_percentage[col]}%")
        # print(f"Match Percentage for {col}: {match_percentage[col]}%")

    # Convert to DataFrame
    result = pd.DataFrame(list(match_percentage.items()), columns=['ColumnName', 'Match Percentage'])
   
    result['Match Percentage'] = result['Match Percentage'].round(2)
    shape = pred.shape
    result['Total Count'] = shape[0]
    actual_df = actual_sorted[columns_to_compare].astype(str)
    llm_df = pred_sorted[columns_to_compare].astype(str)

    # Similarity function
    def similarity(a, b):
        return SequenceMatcher(None, a, b).ratio() * 100

    
    # Initialize summary dictionary for different match categories
    summary = {col: {'100%': 0, '90-100%': 0, '<90': 0} for col in columns_to_compare}

    # DataFrame to store similarity scores for detailed validation table
    comparison_df = pd.DataFrame(index=actual_df.index, columns=columns_to_compare)

    # Populate comparison_df with similarity scores and update summary counts
    for col in columns_to_compare:
        for i in range(len(actual_df)):
            actual_val = actual_df.iloc[i][col]
            llm_val = llm_df.iloc[i][col]
            score = similarity(actual_val, llm_val)
            comparison_df.loc[i, col] = score # Store the similarity score directly

            if score == 100:
                summary[col]['100%'] += 1
            elif score >= 90:
                summary[col]['90-100%'] += 1
            else:
                summary[col]['<90'] += 1

    # Convert summary dictionary to DataFrame
    summary_df = pd.DataFrame.from_dict(summary, orient='index').reset_index()
    summary_df.columns = ['ColumnName', '100%', '90-100%', '<90']
    final_result = pd.merge(result, summary_df, on='ColumnName')

    ## Pivot Table for Summary Evals
    # --- Calculate Overall Summary Metrics for Pivot Table ---
    total_100_matches = 0
    total_90_99_matches = 0
    total_less_89_matches = 0

    # Aggregate counts from the per-column summary
    for col in columns_to_compare:
        total_100_matches += summary[col]['100%']
        total_90_99_matches += summary[col]['90-100%']
        total_less_89_matches += summary[col]['<90']

    # Calculate the total number of individual cell comparisons made across all columns and rows
    total_relevant_entities = len(actual_df) * len(columns_to_compare)

    # Calculate percentages, handling division by zero if no entities to compare
    if total_relevant_entities > 0:
        percent_100 = (total_100_matches / total_relevant_entities) * 100
        percent_90_99 = (total_90_99_matches / total_relevant_entities) * 100
        percent_less_89 = (total_less_89_matches / total_relevant_entities) * 100
    else:
        percent_100 = 0.0
        percent_90_99 = 0.0
        percent_less_89 = 0.0

    # Create the pivot-like summary DataFrame as requested
    summary_pivot_data = {
        "100% Match Entities": [total_100_matches, f"{percent_100:.2f}%"],
        "90-99% Match Entities": [total_90_99_matches, f"{percent_90_99:.2f}%"],
        "<89% Match Entities": [total_less_89_matches, f"{percent_less_89:.2f}%"]
    }
    summary_pivot_df = pd.DataFrame(summary_pivot_data, index=["Count", "% of Total Relevant"])

    # Bold column names and index for the pivot table
    summary_pivot_df_styled = summary_pivot_df.style.set_table_styles([
        # Style for column headers
        {'selector': 'th.col_heading', 'props': [('font-weight', 'bold')]},
        {'selector': 'th.col_heading.level0', 'props': [('font-weight', 'bold')]},

        # Style for index headers
        {'selector': 'th.row_heading', 'props': [('font-weight', 'bold')]},
        {'selector': 'th.row_heading.level0', 'props': [('font-weight', 'bold')]}
    ])

    ## Now, let's create a combined DataFrame for display with styling
    # This display_df will ONLY contain 'Sr. No' and the original column names
    display_df = pd.DataFrame(index=actual_df.index)
    display_df.insert(0, 'Sr. No', range(1, 1 + len(display_df)))

    for col in columns_to_compare:
        # Assign directly to the original column name
        display_df[col] = llm_df[col]
    
    

    # Apply the row-wise styling (green/red) based on (Similarity) values from comparison_df
    # We use a lambda to pass additional arguments to style_validation_table
    styled_display_df_filtered = display_df.style.apply(
        lambda row: style_validation_table(row, comparison_df, columns_to_compare), axis=1
    ).set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold')]}
    ])
    
    total_entities = len(actual_df) # Total number of invoices/rows processed

    # Return the filtered styled DataFrame for the UI
    return summary_pivot_df_styled, styled_display_df_filtered, total_entities, total_relevant_entities


In [208]:
summary , styled, total, relavant = novolex_validate_invoices()

Acutl >>>>>>>>>>>>>>>>> [nan, nan, nan, nan, nan, nan, 4.97, 4.83, 4.77, 4.73, 4.67, 4.59]
Pred  >>>>>>>>>>>>>>>>> [nan, nan, nan, nan, nan, 4.83, 4.97, nan, 4.77, 4.73, 4.67, 4.59]


C:\Users\Anil.Bhallavi\AppData\Local\Temp\ipykernel_19884\1643058467.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred['From'] = pd.to_datetime(pred['From']).dt.strftime('%m/%d/%y')
C:\Users\Anil.Bhallavi\AppData\Local\Temp\ipykernel_19884\1643058467.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred['To'] = pd.to_datetime(pred['To']).dt.strftime('%m/%d/%y')


In [209]:
styled

,Sr. No,Billing Date,Month,From,To,No of Days,Day kWh,Only kWh,Night kWh,Total kWh,kWh per day,kWh %,DUOS Capacity Charge,Excess Capacity Charge,VAT $,City of Cullman Tax $,Alabama State Taxes $,Alabama State Taxes $.1,City of Cullman Tax $.1,Generation/Retail $ amount,Total $ amount (With VAT),Total $ amount (Without VAT),Total $ amount,Blended rate $/kWh (With VAT),Blended rate $/kWh (Without VAT),Blended rate $/kWh
0,1,01/06/25,Jan-25,12/01/24,12/31/24,30,211282.8,nan,65745.5,277028.3,9234.28,2,850.0,110.0,11182.59,nan,nan,nan,nan,nan,67149.6,55967.01000000001,nan,0.24,0.2,nan
1,2,01/30/24,Jan-24,12/30/23,01/30/24,31,nan,1363200.0,nan,1363200.0,43974.19,11,nan,nan,nan,3276.21,5586.7,4.97,2.85,nan,nan,nan,118071.0,nan,nan,0.087
2,3,02/08/25,Feb-25,01/01/25,01/31/25,30,245514.0,nan,72917.5,318431.5,10614.38,3,850.0,nan,12662.12,nan,nan,nan,nan,nan,75972.71,63310.59,nan,0.24,0.2,nan
3,4,02/29/24,Feb-24,01/30/24,02/29/24,30,nan,1584000.0,nan,1584000.0,52800.0,13,nan,nan,nan,3870.52,6418.72,4.83,2.86,nan,nan,nan,139307.0,nan,nan,0.088
4,5,03/06/24,Mar-24,02/01/24,02/29/24,28,242413.4,nan,72594.89,315008.29,11250.3,3,850.0,126.99,11504.37,nan,nan,nan,nan,nan,69080.24,57575.87,nan,0.22,0.18,nan
5,6,03/30/24,Mar-24,02/29/24,03/30/24,30,nan,1540800.0,nan,1540800.0,51360.0,13,nan,nan,nan,4228.0,6919.19,4.77,2.86,nan,nan,nan,152081.0,nan,nan,0.099
6,7,04/30/24,Apr-24,03/30/24,04/30/24,31,nan,1699200.0,nan,1699200.0,54812.9,14,nan,nan,nan,4461.67,7246.34,4.73,2.86,nan,nan,nan,160431.0,nan,nan,0.094
7,8,05/14/24,May-24,04/01/24,04/30/24,29,272378.2,nan,83089.7,355467.9,12257.51,3,850.0,nan,12317.0,nan,nan,nan,nan,nan,73901.95,61584.95,nan,0.21,0.17,nan
8,9,05/30/24,May-24,04/30/24,05/30/24,30,nan,1867200.0,nan,1867200.0,62240.0,15,nan,nan,nan,4907.64,7870.7,4.67,2.86,nan,nan,nan,176367.0,nan,nan,0.094
9,10,06/30/24,Jun-24,05/30/24,06/30/24,31,nan,2006400.0,nan,2006400.0,64722.58,16,nan,nan,nan,5644.07,8901.69,4.59,2.86,nan,nan,nan,202682.0,nan,nan,0.101
